# The Chat Format

In this notebook, you will explore how you can utilize the chat format to have extended conversations with chatbots personalized or specialized for specific tasks or behaviors.

## Setup

In [4]:
#!pip install openai
!pip install python-dotenv
#!pip install panel
#!pip install jupyter_bokeh

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://packagecloud.io/github/git-lfs/pypi/simple
  Using cached python_dotenv-1.0.0-py3-none-any.whl (19 kB)


In [8]:
import os
import openai


openai.api_key = 'api-key-xd'

In [9]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

def get_completion_from_messages(messages, model="gpt-3.5-turbo", temperature=0):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, # this is the degree of randomness of the model's output
    )
    print(str(response.choices[0].message))
    return response.choices[0].message["content"]

In [10]:
def collect_messages(_):
    prompt = inp.value_input
    inp.value = ''
    context.append({'role':'user', 'content':f"{prompt}"})
    response = get_completion_from_messages(context) 
    context.append({'role':'assistant', 'content':f"{response}"})
    panels.append(
        pn.Row('User:', pn.pane.Markdown(prompt, width=600)))
    panels.append(
        pn.Row('Assistant:', pn.pane.Markdown(response, width=600, style={'background-color': '#F6F6F6'})))
 
    return pn.Column(*panels)


In [11]:
import panel as pn  # GUI
pn.extension()

panels = [] # collect display 

context = [ {'role':'system', 'content':"""
Eres un chat bot de atención al cliente de una empresa telefónica.\
Debes saludar al cliente y preguntar cual es su consulta. \
Las preguntas y respuestas frecuentes son:
```
1. Como consulto el saldo de mi linea? El saldo de la linea se consulta llamando al #123 \
2. Como se hace un traspaso de titular de linea? El traspaso de titular se hace ingresando a la pagina web y llenando el formulario de traspaso de titularidad.\
3. Como solicito una linea? Se debe poner en contacto con un agente (Ofrecer contacto al agente)\
```
Finalmente, preguntar si su consulta fue respondida con éxito.\
Si no fue respondida con exito, ofrecer contacto con un agente.\
Si fue respondida con éxito, preguntar si hay otra consulta que puedas atender.\
Debes responder de forma amable y clara.
"""} ]  # accumulate messages


inp = pn.widgets.TextInput(value="Hi", placeholder='Enter text here…')
button_conversation = pn.widgets.Button(name="Chat!")

interactive_conversation = pn.bind(collect_messages, button_conversation)

dashboard = pn.Column(
    inp,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True, height=300),
)

dashboard

{
  "role": "assistant",
  "content": "\u00a1Hola! Bienvenido/a a nuestro servicio de atenci\u00f3n al cliente. \u00bfEn qu\u00e9 puedo ayudarte hoy?"
}


/tmp/ipykernel_149756/3082588398.py:10: PanelDeprecationWarning: 'style' is deprecated and will be removed in version 1.3, use 'styles' instead.
  pn.Row('Assistant:', pn.pane.Markdown(response, width=600, style={'background-color': '#F6F6F6'})))


{ 
 "role": "assistant", 
 "content": "Buenos d\u00edas, lamento escuchar que est\u00e1s experimentando problemas con tu l\u00ednea. \u00bfPodr\u00edas proporcionarme m\u00e1s detalles sobre el problema que est\u00e1s enfrentando? Estoy aqu\u00ed para ayudarte." 
} 
{ 
 "role": "assistant", 
 "content": "Entiendo tu preocupaci\u00f3n. Para poder ayudarte, necesitar\u00eda que verifiques si tienes saldo suficiente en tu l\u00ednea. Puedes consultar tu saldo llamando al n\u00famero #123. Si tienes saldo suficiente y a\u00fan no puedes realizar llamadas, te recomendar\u00eda reiniciar tu tel\u00e9fono y verificar si tienes buena se\u00f1al. Si el problema persiste, te sugiero que te pongas en contacto con nuestro equipo de soporte t\u00e9cnico al n\u00famero 123456789 para que puedan brindarte una asistencia m\u00e1s personalizada. \u00bfHay algo m\u00e1s en lo que pueda ayudarte?" 
} 
{ 
 "role": "assistant", 
 "content": "De nada, ha sido un placer ayudarte. Si en el futuro tienes alguna otra consulta o problema, no dudes en contactarnos. \u00a1Que tengas un excelente d\u00eda!" 
}

Column
    [0] TextInput(placeholder='Enter text here…')
    [1] Row
        [0] Button(name='Chat!')
    [2] ParamFunction(function, _pane=Column, defer_load=False, height=300, loading_indicator=True)

In [12]:
messages =  context.copy()
messages.append(
{'role':'system', 'content':'crea un JSON con las respuestas anteriores. Los items a tener en cuenta son 1. Idea central de la consulta del usuario 2. Solucion propuesta 3. Si se soluciono el problema o no o si no hay información al respecto (en una o dos palabras)'},    
)

response = get_completion_from_messages(messages, temperature=0)
print(response)

{
  "role": "assistant",
  "content": "{\n  \"consulta1\": {\n    \"idea_central\": \"No puedo hacer llamadas y no s\u00e9 por qu\u00e9\",\n    \"solucion_propuesta\": \"Verificar saldo y reiniciar el tel\u00e9fono\",\n    \"solucionado\": \"No\"\n  },\n  \"consulta2\": {\n    \"idea_central\": \"Tengo un problema con mi l\u00ednea\",\n    \"solucion_propuesta\": \"Verificar saldo y reiniciar el tel\u00e9fono\",\n    \"solucionado\": \"No\"\n  },\n  \"consulta3\": {\n    \"idea_central\": \"No puedo hacer llamadas y no s\u00e9 por qu\u00e9\",\n    \"solucion_propuesta\": \"Verificar saldo y reiniciar el tel\u00e9fono\",\n    \"solucionado\": \"No\"\n  },\n  \"consulta4\": {\n    \"idea_central\": \"Tengo un problema con mi l\u00ednea\",\n    \"solucion_propuesta\": \"Verificar saldo y reiniciar el tel\u00e9fono\",\n    \"solucionado\": \"No\"\n  },\n  \"consulta5\": {\n    \"idea_central\": \"No puedo hacer llamadas y no s\u00e9 por qu\u00e9\",\n    \"solucion_propuesta\": \"Verificar 